In [8]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from catboost.utils import eval_metric
from sklearn.model_selection import train_test_split
import time

In [12]:
modelName="/home/alex/projects/1c_ml_regression_diagnostics/cgi-bin/prod_ml_duration_master"
#in_iter_count=
#in_learning_rate=
#in_depth=
in_data_file="/home/alex/projects/1c_ml_regression_diagnostics/cgi-bin/data/master_train_data.csv"
cat_features="field_1,field_2,field_3,field_4,field_5,field_6,field_8,field_11,field_12,field_13"
if len(cat_features)==0:
    cat_features	= None
else:
    cat_features	= cat_features.split(',')
target='field_0'

In [3]:
data_source = pd.read_csv(in_data_file)

In [5]:
fields_count=max(data_source['field'])+1
columns=tuple('field_'+str(i) for i in range(0,int(fields_count)))
data = pd.DataFrame([])
for row in range (0,max(data_source['row'])+1):
    temp=pd.DataFrame(data_source[data_source.row==row]['value']).transpose()
    temp.columns=columns
    data = data.append(temp,ignore_index = True)

data.sort_values(['field_0'], inplace=True)
data.reset_index(drop=True, inplace=True)
data.fillna(0,inplace=True)
data=data.sample(n=len(data))

In [15]:
content=""
X=data.drop('field_0', axis=1)
y=data.field_0
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)			
content+='y: '+str(len(y))+'\n'
content+='y_train: '+str(len(y_train))+'\n'
content+='y_validation: '+str(len(y_validation))+'\n'

#X_test = test_df.drop('field_0', axis=1)
eval_dataset = Pool(data=X_validation,
        label=y_validation,
        cat_features=cat_features)

In [20]:
len(y_train)

6499

In [43]:
content = ""
model1 = CatBoostRegressor(
    cat_features=cat_features,
    depth = 16,
    boost_from_average=True,
    score_function = 'NewtonL2',
    one_hot_max_size = 512,
    langevin = True,
    posterior_sampling=True,
    model_shrink_rate = 1/(2*len(y_train)),
    verbose=True    
    #diffusion_temperature = len(y_train)
    )
#model1.fit(X_train, y_train)
model1.fit(X_train, y_train,use_best_model = True,eval_set=eval_dataset)
pred = model1.predict(X_validation)			
params = model1.get_params()
content+=str(params)+'\n'
content+='\n'+params['loss_function']+' loss: '+ str(eval_metric(y_validation.to_numpy(), pred, params['loss_function']))
content+='\nFitted: '+str(model1.is_fitted())
content+='\nModel score:\n'+str(model1.score(X,y))
content

Learning rate set to 0.068314
0:	learn: 2145.4483551	test: 2124.8806826	best: 2124.8806826 (0)	total: 911ms	remaining: 15m 9s
1:	learn: 2124.4608547	test: 2106.7176481	best: 2106.7176481 (1)	total: 1.78s	remaining: 14m 50s
2:	learn: 2105.0499770	test: 2090.0047777	best: 2090.0047777 (2)	total: 2.77s	remaining: 15m 22s
3:	learn: 2087.6963412	test: 2075.6801596	best: 2075.6801596 (3)	total: 3.77s	remaining: 15m 37s
4:	learn: 2072.8083036	test: 2063.8813108	best: 2063.8813108 (4)	total: 4.73s	remaining: 15m 42s
5:	learn: 2058.8020573	test: 2052.8879156	best: 2052.8879156 (5)	total: 5.74s	remaining: 15m 51s
6:	learn: 2045.7539832	test: 2043.5461003	best: 2043.5461003 (6)	total: 6.74s	remaining: 15m 56s
7:	learn: 2033.9429878	test: 2034.9872108	best: 2034.9872108 (7)	total: 7.79s	remaining: 16m 5s
8:	learn: 2023.0536916	test: 2026.8471335	best: 2026.8471335 (8)	total: 8.81s	remaining: 16m 9s
9:	learn: 2013.2880488	test: 2019.9837990	best: 2019.9837990 (9)	total: 9.88s	remaining: 16m 17s
10:

83:	learn: 1743.1120995	test: 1937.3172657	best: 1937.1734526 (81)	total: 1m 40s	remaining: 18m 13s
84:	learn: 1741.4324474	test: 1936.6839870	best: 1936.6839870 (84)	total: 1m 41s	remaining: 18m 12s
85:	learn: 1739.0877784	test: 1936.7170096	best: 1936.6839870 (84)	total: 1m 42s	remaining: 18m 12s
86:	learn: 1736.3439702	test: 1937.2214438	best: 1936.6839870 (84)	total: 1m 43s	remaining: 18m 9s
87:	learn: 1734.7387095	test: 1937.0237383	best: 1936.6839870 (84)	total: 1m 45s	remaining: 18m 9s
88:	learn: 1733.1737851	test: 1936.4619961	best: 1936.4619961 (88)	total: 1m 46s	remaining: 18m 8s
89:	learn: 1731.7928819	test: 1936.5048941	best: 1936.4619961 (88)	total: 1m 47s	remaining: 18m 7s
90:	learn: 1728.8617745	test: 1936.3892840	best: 1936.3892840 (90)	total: 1m 48s	remaining: 18m 6s
91:	learn: 1726.2532105	test: 1935.7933015	best: 1935.7933015 (91)	total: 1m 49s	remaining: 18m 4s
92:	learn: 1724.1786946	test: 1935.1698443	best: 1935.1698443 (92)	total: 1m 51s	remaining: 18m 4s
93:	lea

165:	learn: 1596.1930042	test: 1925.9319592	best: 1925.6539265 (162)	total: 3m 19s	remaining: 16m 44s
166:	learn: 1594.5850475	test: 1926.0833023	best: 1925.6539265 (162)	total: 3m 21s	remaining: 16m 43s
167:	learn: 1592.7591698	test: 1926.5094107	best: 1925.6539265 (162)	total: 3m 22s	remaining: 16m 42s
168:	learn: 1591.2270847	test: 1926.4502732	best: 1925.6539265 (162)	total: 3m 23s	remaining: 16m 41s
169:	learn: 1589.7389587	test: 1926.4091387	best: 1925.6539265 (162)	total: 3m 24s	remaining: 16m 40s
170:	learn: 1588.3473066	test: 1926.5223483	best: 1925.6539265 (162)	total: 3m 26s	remaining: 16m 40s
171:	learn: 1586.5385719	test: 1926.2734801	best: 1925.6539265 (162)	total: 3m 27s	remaining: 16m 38s
172:	learn: 1585.0759680	test: 1926.3571825	best: 1925.6539265 (162)	total: 3m 28s	remaining: 16m 37s
173:	learn: 1583.7686799	test: 1926.4790634	best: 1925.6539265 (162)	total: 3m 29s	remaining: 16m 36s
174:	learn: 1581.7306956	test: 1926.7741940	best: 1925.6539265 (162)	total: 3m 31s

246:	learn: 1480.9026703	test: 1926.9222039	best: 1925.0640446 (228)	total: 4m 57s	remaining: 15m 5s
247:	learn: 1479.8845272	test: 1926.8142468	best: 1925.0640446 (228)	total: 4m 58s	remaining: 15m 4s
248:	learn: 1478.6571448	test: 1926.5888715	best: 1925.0640446 (228)	total: 4m 59s	remaining: 15m 3s
249:	learn: 1477.1765244	test: 1926.9481037	best: 1925.0640446 (228)	total: 5m	remaining: 15m 1s
250:	learn: 1475.9984618	test: 1926.9935727	best: 1925.0640446 (228)	total: 5m 1s	remaining: 15m 1s
251:	learn: 1475.1398359	test: 1926.6301605	best: 1925.0640446 (228)	total: 5m 3s	remaining: 15m
252:	learn: 1474.4451698	test: 1926.5323851	best: 1925.0640446 (228)	total: 5m 4s	remaining: 14m 58s
253:	learn: 1473.3187537	test: 1926.8204037	best: 1925.0640446 (228)	total: 5m 5s	remaining: 14m 57s
254:	learn: 1471.8443276	test: 1926.6683568	best: 1925.0640446 (228)	total: 5m 6s	remaining: 14m 56s
255:	learn: 1470.8905642	test: 1927.0976161	best: 1925.0640446 (228)	total: 5m 8s	remaining: 14m 55s

327:	learn: 1396.6971266	test: 1930.8902096	best: 1925.0640446 (228)	total: 6m 35s	remaining: 13m 29s
328:	learn: 1395.9864204	test: 1931.0687974	best: 1925.0640446 (228)	total: 6m 36s	remaining: 13m 28s
329:	learn: 1395.0967855	test: 1931.0679026	best: 1925.0640446 (228)	total: 6m 37s	remaining: 13m 27s
330:	learn: 1393.8355414	test: 1931.7163930	best: 1925.0640446 (228)	total: 6m 39s	remaining: 13m 26s
331:	learn: 1392.7005813	test: 1931.9739333	best: 1925.0640446 (228)	total: 6m 40s	remaining: 13m 25s
332:	learn: 1391.7355483	test: 1931.8214515	best: 1925.0640446 (228)	total: 6m 41s	remaining: 13m 24s
333:	learn: 1390.9369113	test: 1932.0433737	best: 1925.0640446 (228)	total: 6m 42s	remaining: 13m 23s
334:	learn: 1390.2093343	test: 1931.9373485	best: 1925.0640446 (228)	total: 6m 43s	remaining: 13m 21s
335:	learn: 1389.2016017	test: 1932.0404863	best: 1925.0640446 (228)	total: 6m 45s	remaining: 13m 20s
336:	learn: 1388.0140263	test: 1931.7548922	best: 1925.0640446 (228)	total: 6m 46s

...

732:	learn: 1103.8554361	test: 1969.1741852	best: 1925.0640446 (228)	total: 14m 44s	remaining: 5m 22s
733:	learn: 1103.0560697	test: 1969.3412538	best: 1925.0640446 (228)	total: 14m 45s	remaining: 5m 20s
734:	learn: 1102.3703453	test: 1969.3566307	best: 1925.0640446 (228)	total: 14m 46s	remaining: 5m 19s
735:	learn: 1101.9337405	test: 1969.5723312	best: 1925.0640446 (228)	total: 14m 47s	remaining: 5m 18s
736:	learn: 1101.3663795	test: 1969.7347056	best: 1925.0640446 (228)	total: 14m 49s	remaining: 5m 17s
737:	learn: 1100.6071735	test: 1969.8003721	best: 1925.0640446 (228)	total: 14m 50s	remaining: 5m 16s
738:	learn: 1100.0422829	test: 1969.9256049	best: 1925.0640446 (228)	total: 14m 51s	remaining: 5m 14s
739:	learn: 1099.6378619	test: 1970.1226694	best: 1925.0640446 (228)	total: 14m 52s	remaining: 5m 13s
740:	learn: 1099.2194740	test: 1970.2525184	best: 1925.0640446 (228)	total: 14m 54s	remaining: 5m 12s
741:	learn: 1098.7807525	test: 1970.4868396	best: 1925.0640446 (228)	total: 14m 55

813:	learn: 1063.4751630	test: 1976.4610427	best: 1925.0640446 (228)	total: 16m 22s	remaining: 3m 44s
814:	learn: 1063.0192429	test: 1976.3792251	best: 1925.0640446 (228)	total: 16m 23s	remaining: 3m 43s
815:	learn: 1062.6294741	test: 1976.3319417	best: 1925.0640446 (228)	total: 16m 25s	remaining: 3m 42s
816:	learn: 1062.2323022	test: 1976.3349795	best: 1925.0640446 (228)	total: 16m 26s	remaining: 3m 40s
817:	learn: 1061.7539737	test: 1976.4308065	best: 1925.0640446 (228)	total: 16m 27s	remaining: 3m 39s
818:	learn: 1061.5696700	test: 1976.4554038	best: 1925.0640446 (228)	total: 16m 28s	remaining: 3m 38s
819:	learn: 1061.1262542	test: 1976.5460236	best: 1925.0640446 (228)	total: 16m 29s	remaining: 3m 37s
820:	learn: 1060.6265862	test: 1976.7206473	best: 1925.0640446 (228)	total: 16m 31s	remaining: 3m 36s
821:	learn: 1059.9538347	test: 1976.9965628	best: 1925.0640446 (228)	total: 16m 32s	remaining: 3m 34s
822:	learn: 1059.3571957	test: 1977.0771548	best: 1925.0640446 (228)	total: 16m 33

894:	learn: 1027.3958025	test: 1983.2117094	best: 1925.0640446 (228)	total: 18m 2s	remaining: 2m 6s
895:	learn: 1026.8631740	test: 1983.0079231	best: 1925.0640446 (228)	total: 18m 3s	remaining: 2m 5s
896:	learn: 1026.5671064	test: 1983.0075649	best: 1925.0640446 (228)	total: 18m 4s	remaining: 2m 4s
897:	learn: 1026.1258919	test: 1983.2111109	best: 1925.0640446 (228)	total: 18m 5s	remaining: 2m 3s
898:	learn: 1025.4866419	test: 1983.3698871	best: 1925.0640446 (228)	total: 18m 6s	remaining: 2m 2s
899:	learn: 1025.0247987	test: 1983.4611510	best: 1925.0640446 (228)	total: 18m 8s	remaining: 2m
900:	learn: 1024.6128998	test: 1983.3982303	best: 1925.0640446 (228)	total: 18m 9s	remaining: 1m 59s
901:	learn: 1024.2599576	test: 1983.4200365	best: 1925.0640446 (228)	total: 18m 10s	remaining: 1m 58s
902:	learn: 1023.8083602	test: 1983.5661740	best: 1925.0640446 (228)	total: 18m 11s	remaining: 1m 57s
903:	learn: 1023.4631505	test: 1983.6877510	best: 1925.0640446 (228)	total: 18m 12s	remaining: 1m 

976:	learn: 994.0915112	test: 1989.4994212	best: 1925.0640446 (228)	total: 19m 41s	remaining: 27.8s
977:	learn: 993.5307512	test: 1989.5100243	best: 1925.0640446 (228)	total: 19m 42s	remaining: 26.6s
978:	learn: 993.2116594	test: 1989.5334557	best: 1925.0640446 (228)	total: 19m 43s	remaining: 25.4s
979:	learn: 992.7936338	test: 1989.7073841	best: 1925.0640446 (228)	total: 19m 44s	remaining: 24.2s
980:	learn: 992.3949903	test: 1989.7029597	best: 1925.0640446 (228)	total: 19m 45s	remaining: 23s
981:	learn: 992.1592702	test: 1989.8889039	best: 1925.0640446 (228)	total: 19m 47s	remaining: 21.8s
982:	learn: 991.9651859	test: 1989.9368569	best: 1925.0640446 (228)	total: 19m 48s	remaining: 20.5s
983:	learn: 991.7364264	test: 1989.8853379	best: 1925.0640446 (228)	total: 19m 49s	remaining: 19.3s
984:	learn: 991.2604426	test: 1989.9825789	best: 1925.0640446 (228)	total: 19m 50s	remaining: 18.1s
985:	learn: 990.6981038	test: 1990.1647822	best: 1925.0640446 (228)	total: 19m 51s	remaining: 16.9s
98

"{'boost_from_average': True, 'posterior_sampling': True, 'langevin': True, 'model_shrink_rate': 7.693491306354824e-05, 'score_function': 'NewtonL2', 'cat_features': ['field_1', 'field_2', 'field_3', 'field_4', 'field_5', 'field_6', 'field_8', 'field_11', 'field_12', 'field_13'], 'one_hot_max_size': 1024, 'verbose': True, 'loss_function': 'RMSE', 'depth': 16}\n\nRMSE loss: [1924.9924219889285]\nFitted: True\nModel score:\n0.43882909654918456"

In [36]:
content

"{'boost_from_average': True, 'posterior_sampling': True, 'langevin': True, 'model_shrink_rate': 7.693491306354824e-05, 'score_function': 'NewtonL2', 'cat_features': ['field_1', 'field_2', 'field_3', 'field_4', 'field_5', 'field_6', 'field_8', 'field_11', 'field_12', 'field_13'], 'one_hot_max_size': 512, 'verbose': False, 'loss_function': 'RMSE', 'depth': 16}\n\nRMSE loss: [1924.9924219889285]\nFitted: True\nModel score:\n0.43882909654918456"

In [33]:
content

"{'boost_from_average': True, 'posterior_sampling': True, 'model_shrink_rate': 7.693491306354824e-05, 'score_function': 'NewtonL2', 'cat_features': ['field_1', 'field_2', 'field_3', 'field_4', 'field_5', 'field_6', 'field_8', 'field_11', 'field_12', 'field_13'], 'one_hot_max_size': 512, 'verbose': False, 'loss_function': 'RMSE'}\n\nRMSE loss: [1915.9784853156034]\nFitted: True\nModel score:\n0.41160723013528167"

In [17]:
content

"y: 8666\ny_train: 6499\ny_validation: 2167\n{'boost_from_average': True, 'score_function': 'NewtonL2', 'cat_features': ['field_1', 'field_2', 'field_3', 'field_4', 'field_5', 'field_6', 'field_8', 'field_11', 'field_12', 'field_13'], 'one_hot_max_size': 512, 'loss_function': 'RMSE'}\n\nRMSE loss: [1916.8742743427579]\nFitted: True\nModel score:\n0.3783976891299444"